# Basic training and fine-tuning with RAGatouille

In this quick example, we'll use the `RAGTrainer` magic class to demonstrate how to very easily fine-tune an existing ColBERT model, or train one from any BERT/RoBERTa-like model (to [train one for a previously unsupported language like Japanese](https://huggingface.com/bclavie/jacolbert), for example!)

First, we'll create an instance of `RAGtrainer`. We need to give it two arguments: the `model_name` we want to give to the model we're training, and the `pretrained_model_name` of the base model. This can either be a local path, or the name of a model on the HuggingFace Hub. If you're training for a language other than English, you should also include a `language_code` two-letter argument (PLACEHOLDER ISO) so we can get the relevant processing utils!

The trainer will auto-detect whether it's an existing ColBERT model or a BERT base model, and will set itself up accordingly!

Whether we're training from scratch or fine-tuning doesn't matter, all the steps are the same. For this example, let's fine-tune ColBERTv2:

In [1]:
from ragatouille import RAGTrainer
trainer = RAGTrainer(model_name="GhibliColBERT", pretrained_model_name="colbert-ir/colbertv2.0", language_code="en")

To train retrieval models like colberts, we need training triplets: queries, positive passages, and negative passages for each query.

In the next tutorial, we'll see [how to generate synthetic queries when we don't have any annotated passage]. For this tutorial, we'll assume that we have queries and relevant passages, but that we're lacking negative ones (because it's not an information we gather from our users).

Let's assume our corpus is the same as the one we [used for our example about indexing an searching](PLACEHOLDER): Hayao Miyazaki's wikipedia page. Let's first fetch the content from Wikipedia:

In [2]:
import requests

def get_wikipedia_page(title: str):
    """
    Retrieve the full text content of a Wikipedia page.
    
    :param title: str - Title of the Wikipedia page.
    :return: str - Full text content of the page as raw string.
    """
    # Wikipedia API endpoint
    URL = "https://en.wikipedia.org/w/api.php"

    # Parameters for the API request
    params = {
        "action": "query",
        "format": "json",
        "titles": title,
        "prop": "extracts",
        "explaintext": True,
    }

    # Custom User-Agent header to comply with Wikipedia's best practices
    headers = {
        "User-Agent": "RAGatouille_tutorial/0.0.1 (ben@clavie.eu)"
    }

    response = requests.get(URL, params=params, headers=headers)
    data = response.json()

    # Extracting page content
    page = next(iter(data['query']['pages'].values()))
    return page['extract'] if 'extract' in page else None

In [3]:
my_full_corpus = [get_wikipedia_page("Hayao_Miyazaki")]
my_full_corpus += [get_wikipedia_page("Studio_Ghibli")]
my_full_corpus += [get_wikipedia_page("Toei_Animation")]

We're also some Toei Animation content -- it helps to have things in our corpus that aren't directly relevant to our queries but are likely to cover similar topics, so we can get some more adjacent negative examples.

The documents are very long, so let's use a `CorpusProcessor` to split them into chunks of around 256 tokens:

In [4]:
from ragatouille.data import CorpusProcessor, llama_index_sentence_splitter

corpus_processor = CorpusProcessor(document_splitter_fn=llama_index_sentence_splitter)
documents = corpus_processor.process_corpus(my_full_corpus, chunk_size=256)

Now that we have a corpus of documents, let's generate fake query-relevant passage pair. Obviously, you wouldn't want that in the real world, but that's the topic of the next tutorial:

In [5]:
import random

queries = ["What manga did Hayao Miyazaki write?",
           "which film made ghibli famous internationally",
           "who directed Spirited Away?",
           "when was Hikotei Jidai published?",
           "where's studio ghibli based?",
           "where is the ghibli museum?"
] * 3
pairs = []

for query in queries:
    fake_relevant_docs = random.sample(documents, 10)
    for doc in fake_relevant_docs:
        pairs.append((query, doc))

Here, we have created pairs.It's common for retrieval training data to be stored in a lot of different ways: pairs of [query, positive], pairs of [query, passage, label], triplets of [query, positive, negative], or triplets of [query, list_of_positives, list_of_negatives]. No matter which format your data's in, you don't need to worry about it: RAGatouille will generate ColBERT-friendly triplets for you, and export them to disk for easy `dvc` or `wandb` data tracking.

Speaking of, let's process the data so it's ready for training. `RAGTrainer` has a `prepare_training_data` function, which will perform all the necessary steps. One of the steps it performs is called **hard negative mining**: that's searching the full collection of documents (even those not linked to a query) to retrieve passages that are semantically close to a query, but aren't actually relevant. Using those to train retrieval models has repeatedly been shown to greatly improve their ability to find actually relevant documents, so it's a very important step! 

RAGatouille handles all of this for you. By default, it'll fetch 10 negative examples per query, but you can customise this with `num_new_negatives`. You can also choose not to mine negatives and just sample random examples to speed up things, this might lower performance but will run done much quicker on large volumes of data, just set `mine_hard_negatives` to `False`. If you've already mined negatives yourself, you can set `num_new_negatives` to 0 to bypass this entirely.

In [6]:
trainer.prepare_training_data(raw_data=pairs, data_out_path="./data/", all_documents=my_full_corpus, num_new_negatives=10, mine_hard_negatives=True)

Loading Hard Negative SimpleMiner dense embedding model BAAI/bge-small-en-v1.5...
Building hard negative index for 93 documents...
All documents embedded, now adding to index...
save_index set to False, skipping saving hard negative index
Hard negative index generated
mining
mining
mining
mining
mining
mining


'./data/'

Our training data's now fully processed and saved to disk in `data_out_path`! We're now ready to begin training our model with the `train` function. `train` takes many arguments, but the set of default is already fairly strong!

Don't be surprised you don't see an `epochs` parameter here, ColBERT will train until it either reaches `maxsteps` or has seen the entire training data once (a full epoch), this is by design!

In [8]:

trainer.train(batch_size=32,
              nbits=4, # How many bits will the trained model use when compressing indexes
              maxsteps=500000, # Maximum steps hard stop
              use_ib_negatives=True, # Use in-batch negative to calculate loss
              dim=128, # How many dimensions per embedding. 128 is the default and works well.
              learning_rate=5e-6, # Learning rate, small values ([3e-6,3e-5] work best if the base model is BERT-like, 5e-6 is often the sweet spot)
              doc_maxlen=256, # Maximum document length. Because of how ColBERT works, smaller chunks (128-256) work very well.
              use_relu=False, # Disable ReLU -- doesn't improve performance
              warmup_steps="auto", # Defaults to 10%
             )


#> Starting...
nranks = 1 	 num_gpus = 1 	 device=0
{
    "query_token_id": "[unused0]",
    "doc_token_id": "[unused1]",
    "query_token": "[Q]",
    "doc_token": "[D]",
    "ncells": null,
    "centroid_score_threshold": null,
    "ndocs": null,
    "load_index_with_mmap": false,
    "index_path": null,
    "nbits": 4,
    "kmeans_niters": 20,
    "resume": false,
    "similarity": "cosine",
    "bsize": 32,
    "accumsteps": 1,
    "lr": 5e-6,
    "maxsteps": 500000,
    "save_every": 0,
    "warmup": 0,
    "warmup_bert": null,
    "relu": false,
    "nway": 2,
    "use_ib_negatives": true,
    "reranker": false,
    "distillation_alpha": 1.0,
    "ignore_scores": false,
    "model_name": "GhibliColBERT",
    "query_maxlen": 32,
    "attend_to_mask_tokens": false,
    "interaction": "colbert",
    "dim": 128,
    "doc_maxlen": 256,
    "mask_punctuation": true,
    "checkpoint": "colbert-ir\/colbertv2.0",
    "triples": "data\/triples.train.colbert.jsonl",
    "collection": "data\

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "



#> LR will use 0 warmup steps and linear decay over 500000 steps.

#> QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
#> Input: . What manga did Hayao Miyazaki write?, 		 True, 		 None
#> Output IDs: torch.Size([32]), tensor([  101,     1,  2054,  8952,  2106, 10974,  7113,  2771,  3148, 18637,
         4339,  1029,   102,   103,   103,   103,   103,   103,   103,   103,
          103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
          103,   103], device='cuda:0')
#> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')

				 4.857693195343018 8.471616744995117
#>>>    16.0 20.45 		|		 -4.449999999999999
[Jan 03, 17:32:12] 0 13.329309463500977
				 5.369233131408691 9.21822452545166
#>>>    15.58 20.6 		|		 -5.020000000000001
[Jan 03, 17:32:12] 1 13.330567611694336
				 9.961652755737305 15.010833740234375
#>>>    10.04 19.99 		|		 -9

And you're now done training! Your model is saved at the path it outputs, with the final checkpoint always being in the `.../checkpoints/colbert` path, and intermediate checkpoints saved at `.../checkpoints/colbert-{N_STEPS}`.

You can now use your model by pointing at its local path, or upload it to the huggingface hub to share it with the world!